Para entrar no modo apresentação, execute a seguinte célula e pressione `-`

In [ ]:
%reload_ext slide

<span class="notebook-slide-start"/>

# APIs do GitHub (v3)

Este notebook apresenta os seguintes tópicos:

- [APIs do GitHub](#APIs-do-GitHub)
- [Autenticação](#Autentica%C3%A7%C3%A3o)
- [API v3](#API-v3)
- [Exercício 6](#Exerc%C3%ADcio-6)
- [Exercício 7](#Exerc%C3%ADcio-7)
- [Exercício 8](#Exerc%C3%ADcio-8)

## APIs do GitHub

Como o GitHub oferece APIs para obter informações de repositórios, usá-las em geral é melhor do que fazer crawling.

O GitHub possui duas versões estáveis de APIs:

- REST API v3: https://developer.github.com/v3/
- GraphQL API v4: https://developer.github.com/v4/

A forma de usar cada API é diferente e a taxa de requisições permitidas também é. Neste minicurso, usaremos requests para acessar ambas as APIs, mas existem bibliotecas prontas (como a PyGitHub para a v3) que fazem o acesso.

### Autenticação

Para usar qualquer uma das APIs, é necessário gerar um token de autenticação no GitHub seguindo os seguintes passos.

Primeiro, vá em configurações da conta.

<img src="images/github1.png" alt="Página inicial do GitHub" width="auto"/>

Em seguida, abra configurações de desenvolvedor.

<img src="images/github2.png" alt="Página de Configurações do Usuário" width="auto"/>

Abra "Personal access tokens" e clique em "Generate new token". 

<img src="images/github3.png" alt="Página de Tokens de Acesso Pessoal" wi3dth="auto"/>

Escolha as permissões que você deseja no token. 

<img src="images/github4.png" alt="Página de Criação de Token de Acesso Pessoal" width="auto"/>

Copie o token gerado para algum lugar seguro. Para o minicurso, eu copiei o meu token para `~/githubtoken.txt` e vou carregá-lo para a variável `token` a seguir. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
from ipywidgets import FileUpload, interact
@interact(files=FileUpload())
def set_token(files={}):
    global token
    if files:
        for key, values in files.items():
            token = values['content'].decode("utf-8").strip()
            print("Token Loaded!")

## API v3

Com o token em mãos, podemos começar a usa a API v3. O acesso a API do GitHub é feito a https://api.github.com. Portanto, precisamos mudar o site de nosso servidor de proxy. Para isso, podemos fechar e reiniciar da seguinte forma:

```bash
python proxy.py https://api.github.com/
```

Inicialmente, vamos fazer uma requisição para verificar se a autenticação funciona e para vermos nosso limite de requisições. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
import requests 
SITE = "http://localhost:5000/" # ou https://api.github.com
def token_auth(request):
    request.headers["User-Agent"] = "Minicurso" # Necessário
    request.headers["Authorization"] = "token {}".format(token)
    return request
response = requests.get(SITE, auth=token_auth)
response.status_code

Resultado 200 - a autenticação funcionou. <span class="notebook-slide-scroll" data-position="-1"/>

O limite de acesso vem definido no header. <span class="notebook-slide-extra" data-count="3"/>

In [ ]:
response.headers["X-RateLimit-Limit"]

In [ ]:
response.headers["X-RateLimit-Remaining"]

In [ ]:
response.headers["X-RateLimit-Reset"]

O retorno da API v3 é sempre um JSON. O acesso a https://api.github.com retorna as URLS válidas da API. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
response.json()

Vamos ver o que a API tem sobre algum repositório.

Primeiro precisamos ver qual URL usar. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
_['repository_url']

Em seguida, fazemos a requisição para saber o que tem no repositorio `gems-uff/sapos`. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
response = requests.get(SITE + "repos/gems-uff/sapos", auth=token_auth)
response.status_code

In [ ]:
data = response.json()
data

O resultado tem diversos resultados e URLs para pegar mais informações. Vamos pegar algumas informações diretas interessantes. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
print("Estrelas:", data["stargazers_count"])
print("Forks:", data["forks"])
print("Watchers:", data["subscribers_count"])
print("Issues abertas:", data["open_issues"])
print("Linguagem:", data["language"])

Se quisermos saber quem são os colaboradores do projeto e quais são as issues existentes, podemos obter as respectivas URLs. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
print("Colaboradores:", data["contributors_url"])
print("Issues:", data["issues_url"])

Agora podemos obter a lista de colaboradores. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
response = requests.get(SITE + "repos/gems-uff/sapos/contributors", auth=token_auth)
response.status_code

In [ ]:
data = response.json()
data

### Gráfico de Barras

A partir desta lista, podemos fazer um gráfico de barras de contribuições. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
contributions = {x["login"]: x["contributions"] for x in data}
contributions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
labels, values = zip(*contributions.items())
indexes = np.arange(len(labels))
width = 1
plt.barh(indexes, values, width)
plt.yticks(indexes, labels)
plt.show()

Nesse código:

- Importamos `matplotlib` e `numpy` para gerar o gráfico
- Chamamos `%matplotlib inline` para permitir a visualização da figura
- Separamos o dicionário `contributions` em duas listas de `labels` e `values`
- Criamos um `arange` de índices
- Criamos o gráfico de barras horizontal, usando `barh`
- Definimos os ticks de y como sendo os `labels` extraídos anteriormente

Podemos acessar também a url de issues. <span class="notebook-slide-extra" data-count="3"/>

In [ ]:
response = requests.get(SITE + "repos/gems-uff/sapos/issues", auth=token_auth)
response.status_code

In [ ]:
data = response.json()
data

In [ ]:
len(data)

Por padrão, a API retorna 30 itens por página. Dessa forma. a lista retornou apenas a primeira página de issues. <span class="notebook-slide-position" data-count="-1"/>

Podemos acessar a segunda página com o parâmetro `?page=2`. <span class="notebook-slide-extra" data-count="3"/>

In [ ]:
response = requests.get(SITE + "repos/gems-uff/sapos/issues?page=2", auth=token_auth)
response.status_code

In [ ]:
data2 = response.json()
data2

In [ ]:
len(data2)

Podemos formar uma lista com todas as issues abertas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
open_issues = data + data2

Essas são apenas as issues abertas. Para pegarmos as issues fechadas, precisamos definir `state=closed`. Podemos aproveitar e definir também `per_page=100` (limite máximo) e fazer um código para pegar todas as páginas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
should_continue = True
page = 1
closed_issues = []
while should_continue:
    response = requests.get(SITE + "repos/gems-uff/sapos/issues?page={}&per_page=100&state=closed".format(page), auth=token_auth)
    if response.status_code != 200:
        print("Fail:", response.status_code)
        break
    data = response.json()
    closed_issues += data
    if len(data) < 100:
        should_continue = False
    page += 1
len(closed_issues), page - 1

Foram encontradas 262 issues em 3 páginas. <span class="notebook-slide-scroll" data-position="-1"/>

Agora podemos fazer um gráfico que mostre a evolução de issues abertas ao longo do tempo. 

Para fazer esse gráfico, primeiro precisamos combinar as issues e descobrir qual foi a data da issue mais antiga. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
import dateutil.parser

all_issues = open_issues + closed_issues
oldest_issue = min(
    all_issues, 
    key=lambda x: dateutil.parser.parse(x["created_at"])
)

oldest_date = dateutil.parser.parse(oldest_issue["created_at"])
oldest_date

A partir desta data, podemos criar um range de dias até hoje para ser o nosso índice do gráfico e um array de zeros do `numpy` para acumularmos a quantidade de issues abertas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
from datetime import datetime, timezone
today = datetime.now(timezone.utc)
delta = today - oldest_date
days = delta.days
print(days)
indexes = np.arange(days)
values = np.zeros(days)

Podemos percorrer todas as issues abertas, incrementando `values` do período em que elas foram abertas até hoje. E podemos percorrer todas as issues fechadas incrementando `values` do período em que elas foram abertas até o período em que elas foram fechadas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
for issue in open_issues:
    created_at = dateutil.parser.parse(issue["created_at"])
    created_at_index = (created_at - oldest_date).days
    values[created_at_index:] += 1
    
for issue in closed_issues:
    created_at = dateutil.parser.parse(issue["created_at"])
    created_at_index = (created_at - oldest_date).days
    
    closed_at = dateutil.parser.parse(issue["closed_at"])
    closed_at_index = (closed_at - oldest_date).days
    values[created_at_index:closed_at_index] += 1

Já é possível plotar o gráfico desta forma, mas o entendimento dos eixos ainda não é o ideal. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
plt.plot(indexes, values)

Precisamos definir quais são os anos no eixo x. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
from math import ceil
labels = [datetime(2013 + i, 1, 1, tzinfo=timezone.utc) for i in range(ceil(delta.days / 365))]
label_indexes = [(label - oldest_date).days for label in labels]
label_years = [label.year for label in labels]
plt.xticks(label_indexes, label_years)
plt.plot(indexes, values)
plt.show()

Também podemos definir o que é cada eixo. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
plt.xticks(label_indexes, label_years)
plt.xlabel("Time")
plt.ylabel("Open Issues")
plt.plot(indexes, values)
plt.show()

Issues podem ter diversos labels. Agora vamos fazer um gráfico que mostre barras estacadas com a evolução de cada tipo de issue. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
from collections import defaultdict
values = defaultdict(lambda: np.zeros(days))

for issue in open_issues:
    created_at = dateutil.parser.parse(issue["created_at"])
    created_at_index = (created_at - oldest_date).days
    for label in issue["labels"]:
        values[label["name"]][created_at_index:] += 1
    if not issue["labels"]:
        values["no-label"][created_at_index:] += 1
        
for issue in closed_issues:
    created_at = dateutil.parser.parse(issue["created_at"])
    created_at_index = (created_at - oldest_date).days
    
    closed_at = dateutil.parser.parse(issue["closed_at"])
    closed_at_index = (closed_at - oldest_date).days
    for label in issue["labels"]:
        values[label["name"]][created_at_index:closed_at_index] += 1
    if not issue["labels"]:
        values["no-label"][created_at_index:closed_at_index] += 1
        


In [ ]:
bottom = np.zeros(days)
legend_color = []
legend_text = []
for label, yvalues in values.items():
    if not label[0].isdigit(): # Exclui tags de versões
        ax = plt.bar(indexes, yvalues, 1,
                     bottom=bottom)
        legend_color.append(ax[0])
        bottom += yvalues
        legend_text.append(label)
        
plt.xticks(label_indexes, label_years)
plt.xlabel("Time")
plt.ylabel("Open Issues By Type")
plt.legend(legend_color, legend_text)
plt.show()

## Exercício 6

Crie um gráfico de linhas que mostre apenas issues do tipo bug. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
...

plt.xlabel("Time")
plt.ylabel("Open Bug Issues")
plt.show()

## Exercício 7

Crie um gráfico de barras para mostrar a participação de usuários em cada issue. Considere o atributo `user`. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
...

In [ ]:
...

plt.xlabel("Time")
plt.ylabel("Open Issues By User")
plt.legend(
    legend_color, legend_text,
    bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",
    mode="expand", borderaxespad=0, ncol=2
)
plt.show()

## Exercício 8

Filtre o gráfico do total de issues abertas para mostrar apenas o ano 2014. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
yfirst = datetime(2014, 1, 1, tzinfo=timezone.utc)
ylast = datetime(2015, 1, 1, tzinfo=timezone.utc)

deltadays = (ylast - yfirst).days
values = np.zeros(deltadays)
indexes = np.arange(deltadays)

...

In [ ]:
labels = [datetime(2014, i + 1, 1, tzinfo=timezone.utc) for i in range(12)]
label_indexes = [(label - yfirst).days for label in labels]
label_years = [label.month for label in labels]
plt.xticks(label_indexes, label_years)
plt.plot(indexes, values)
plt.show()

Continua: [7.API.v4.ipynb](7.API.v4.ipynb)

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;


&nbsp;

&nbsp;

&nbsp;

